# Overviews

This notebook demos retrieving overview data series

In [ ]:
%matplotlib widget

import datetime as dt
import logging

import httpx
import matplotlib.pyplot as plt
import shapely
import sqlmodel

from arpav_ppcv import (
    config,
    database,
    db,
    operations,
    timeseries,
)
from arpav_ppcv.schemas import (
    static,
)

logging.basicConfig(level=logging.DEBUG)
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("httpcore").setLevel(logging.WARNING)
logging.getLogger("matplotlib").setLevel(logging.WARNING)

http_client = httpx.Client()
settings = config.get_settings()
session = sqlmodel.Session(database.get_engine(settings))

forecast_overview_series_configuration = db.get_forecast_overview_series_configuration_by_identifier(
    session,
    "overview-forecast-tas-absolute-annual"
)
observation_overview_series_configuration = db.get_observation_overview_series_configuration_by_identifier(
    session,
    "overview-historical-tas-absolute-annual"
)
print(forecast_overview_series_configuration.identifier)
print(observation_overview_series_configuration.identifier)

internal_forecast_overview_series = db.generate_forecast_overview_series_from_configuration(forecast_overview_series_configuration)
internal_observation_overview_series = db.generate_observation_overview_series_from_configuration(observation_overview_series_configuration)
print([fos.identifier for fos in internal_forecast_overview_series])
print(internal_observation_overview_series.identifier)

In [ ]:
overview_series_result = timeseries.get_overview_time_series(
    settings=settings,
    session=session,
    processing_methods=[
        static.CoverageTimeSeriesProcessingMethod.NO_PROCESSING,
        static.CoverageTimeSeriesProcessingMethod.MOVING_AVERAGE_11_YEARS,
        static.CoverageTimeSeriesProcessingMethod.LOESS_SMOOTHING,
    ],
    include_uncertainty=True
)

In [ ]:
for idx, osr in enumerate(overview_series_result):
    print(idx, osr.identifier)

In [ ]:
fig = plt.figure()

In [ ]:
for pd_series in overview_series_result:
    pd_series.data_.plot()
plt.legend()